<a href="https://colab.research.google.com/github/MariaZharova/test_rec_systems/blob/main/Compare_LightFM_and_Microsoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix

In [2]:
! pip install lightfm recommenders scrapbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 10.8 MB/s 
     |████████████████████████████████| 335 kB 8.7 MB/s 
     |████████████████████████████████| 197 kB 57.6 MB/s 
     |████████████████████████████████| 355 kB 61.6 MB/s 
     |████████████████████████████████| 12.4 MB 54.0 MB/s 
     |████████████████████████████████| 61 kB 10.0 MB/s 
     |████████████████████████████████| 11.8 MB 62.2 MB/s 
     |████████████████████████████████| 636 kB 59.6 MB/s 
     |████████████████████████████████| 4.4 MB 46.1 MB/s 
     |████████████████████████████████| 388 kB 64.4 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.1 MB/s 
     |████████████████████████████████| 106 kB 71.7 MB/s 
     |████████████████████████████████| 132 kB 77.0 MB/s 
     |████████████████████████████████| 423 kB 72.7 MB/s 
  Created wheel for lightfm: fi

In [4]:
import re
import sys
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import pandas as pd 

from collections import defaultdict
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.datasets.split_utils import filter_k_core

# Transformer Based Models
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT

# Sampler for sequential prediction
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet

# Evaluation
from recommenders.evaluation.python_evaluation import precision_at_k

In [24]:
data = pd.read_csv('clicks_cleared.csv')
print(data.shape)
data.head()

(1504242, 11)


,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo
0,0,2022-06-29 01:59:29,e4ae2d7f9eab4cb7,87603527,android,OpenOfferScreen,FavoritesScreen,274235341.0,2022-06-29,274235341.0,2.0
1,1,2022-06-29 01:59:29,e4ae2d7f9eab4cb7,87603527,android,OpenOfferScreen,FavoritesScreen,274235341.0,2022-06-29,274235341.0,2.0
2,2,2022-06-29 01:59:29,e4ae2d7f9eab4cb7,87603527,android,OpenOfferScreen,FavoritesScreen,274235341.0,2022-06-29,274235341.0,2.0
3,3,2022-06-28 23:45:45,1ff927e168b94e02,87603527,android,OpenOfferScreen,RecommendationsScreen,274235341.0,2022-06-29,274235341.0,2.0
4,4,2022-06-28 23:45:45,1ff927e168b94e02,87603527,android,OpenOfferScreen,RecommendationsScreen,274235341.0,2022-06-29,274235341.0,2.0


### Prepare data

In [25]:
# add action column
data['action'] = 1

# encode, start with 0
offer_encoder = {off: ind for ind, off in enumerate(data['offer_id'].unique())}
data['offer_id_enc'] = data['offer_id'].map(offer_encoder)
uid_encoder = {uid: ind for ind, uid in enumerate(data['uid'].unique())}
data['uid_enc'] = data['uid'].map(uid_encoder)

In [26]:
# sort by time
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.sort_values(by='timestamp', inplace=True)
print(data.shape)
data.head(10)

(1504242, 14)


,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo,action,offer_id_enc,uid_enc
834751,834751,2022-06-27 21:00:00,2241fefec0494315,398271fb-efcf-4875-bce8-69e550e89b36,android,OpenOfferScreen,MapScreen,272813805.0,2022-06-28,272813805.0,2.0,1,10044,5040
834750,834750,2022-06-27 21:00:00,2241fefec0494315,398271fb-efcf-4875-bce8-69e550e89b36,android,OpenOfferScreen,MapScreen,272813805.0,2022-06-28,272813805.0,2.0,1,10044,5040
360762,360762,2022-06-27 21:00:00,f522e557db8b4679,75545312,ios,OpenOfferScreen,PersonalFeedScreen,275048289.0,2022-06-28,275048289.0,2.0,1,3579,5995
360761,360761,2022-06-27 21:00:00,f522e557db8b4679,75545312,ios,OpenOfferScreen,PersonalFeedScreen,275048289.0,2022-06-28,275048289.0,2.0,1,3579,5995
834749,834749,2022-06-27 21:00:00,2241fefec0494315,398271fb-efcf-4875-bce8-69e550e89b36,android,OpenOfferScreen,MapScreen,272813805.0,2022-06-28,272813805.0,2.0,1,10044,5040
834748,834748,2022-06-27 21:00:00,2241fefec0494315,398271fb-efcf-4875-bce8-69e550e89b36,android,OpenOfferScreen,MapScreen,272813805.0,2022-06-28,272813805.0,2.0,1,10044,5040
360760,360760,2022-06-27 21:00:00,f522e557db8b4679,75545312,ios,OpenOfferScreen,PersonalFeedScreen,275048289.0,2022-06-28,275048289.0,2.0,1,3579,5995
1313747,1313747,2022-06-27 21:00:01,8eb54de70ee64369,92650098,android,OpenOfferScreen,MapScreen,273499093.0,2022-06-28,273499093.0,2.0,1,19830,767
1313748,1313748,2022-06-27 21:00:01,8eb54de70ee64369,92650098,android,OpenOfferScreen,MapScreen,273499093.0,2022-06-28,273499093.0,2.0,1,19830,767
1313749,1313749,2022-06-27 21:00:01,8eb54de70ee64369,92650098,android,OpenOfferScreen,MapScreen,273499093.0,2022-06-28,273499093.0,2.0,1,19830,767


# LightFM

In [14]:
# create train and test sets
#print(data.iloc[1800000])
del_ind = 1600000
data_train = data.iloc[: data.shape[0]//3*2, :]
data_test = data.iloc[data.shape[0]//3*2 :, :]

data_train = data_train.groupby(by=['uid_enc', 'offer_id_enc']).count().reset_index()
data_test = data_test.groupby(by=['uid_enc', 'offer_id_enc']).count().reset_index()
data_train

,uid_enc,offer_id_enc,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo,action
0,0,0,9,9,9,9,9,9,9,9,9,9,9,9
1,0,101,10,10,10,10,10,10,10,10,10,10,10,10
2,0,176,4,4,4,4,4,4,4,4,4,4,4,4
3,0,325,3,3,3,3,3,3,3,3,3,3,3,3
4,0,330,3,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226597,40182,13029,2,2,2,2,2,2,2,2,2,2,2,2
226598,40183,13030,2,2,2,2,2,2,2,2,2,2,2,2
226599,40184,13037,2,2,2,2,2,2,2,2,2,2,2,2
226600,40185,13037,2,2,2,2,2,2,2,2,2,2,2,2


In [15]:
# create sparse matrices
csr_user_item_train = csr_matrix((data_train['action'], (data_train['uid_enc'], data_train['offer_id_enc'])))

In [16]:
# make test and train datasets
train_users = set(data_train['uid_enc'].values)
train_items = set(data_train['offer_id_enc'].values)


data_test = data_test[(data_test['uid_enc'].isin(train_users)) & 
                      (data_test['offer_id_enc'].isin(train_items))]

# make test dict
test_dict = {}
for user_id in data_test['uid_enc'].drop_duplicates().values:
    test_dict[user_id] = list(data_test[data_test['uid_enc'] == user_id]['offer_id_enc'])

In [17]:
# instantiate and train the model
model = LightFM(loss='warp', no_components=70)
model.fit(csr_user_item_train, epochs=100, num_threads=7, verbose=True) # verbose

Epoch: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


In [18]:
all_items = list(train_items)
#all_items = data['id_enc'].values
def get_predictions(user_id, topk=100):
    preds = model.predict(user_id, all_items)
    inds = np.argsort(preds)[::-1]
    return inds[:topk]

preds = {}

for user_id in data_test['uid_enc'].drop_duplicates().values:
    pred_user = get_predictions(int(user_id), topk=5)
    preds[user_id] = pred_user.tolist()

### Evaluate LightFM

In [19]:
from sklearn.utils import column_or_1d
from sklearn.utils.multiclass import type_of_target

def precision_at_k(y_true, y_score, k):
    """
        INPUT:
        y_true - list of true test values for one user
        y_score - list of predictions values foro one user
        k - level of precision
        OUTPUT:
        ans - precision at k :)
    """
    ans = 0
    counter = 0
    for row in y_true:
        counter += 1
        if counter > k:
            return ans
        if row in y_score:
            ans += 1 / k
    return ans

prec_fin = 0
for ind, val in test_dict.items():
    #print(val,  preds[ind])
    prec_fin += precision_at_k(val, preds[ind], 5)
    
prec_fin /= len(test_dict)

In [20]:
prec_fin

0.018181818181818184

# SASRec

In [27]:
# create .txt file for input to model
data[['offer_id_enc',	'uid_enc']].to_csv('out.txt', sep="\t", header=False, index=False)

In [28]:
# create special data format for SAS
dataS = SASRecDataSet(filename='out.txt', col_sep='\t')
# split into train, test and validation
dataS.split()

In [106]:
def data_partition_with_time(dataS):
    # assume user/item index starting from 1
    """ 
        нужно изменить user_train, user_valid, user_test
    """
    f = open(dataS.filename, "r")
    for line in f:
        u, i = line.rstrip().split(dataS.col_sep)
        u = int(u)
        i = int(i)
        dataS.usernum = max(u, dataS.usernum)
        dataS.itemnum = max(i, dataS.itemnum)
        dataS.User[u].append(i)

    del_ind = 1600000
    data_train = data.iloc[:del_ind, :]
    data_test = data.iloc[del_ind:, :]

    train_users = set(data_train['uid_enc'].values)
    train_items = set(data_train['offer_id_enc'].values)
    data_test = data_test[(data_test['uid_enc'].isin(train_users)) & 
                          (data_test['offer_id_enc'].isin(train_items))]

    for user in train_users:
        if data[data['uid_enc'] == user]['action'].iloc[0] < 3:
            dataS.user_train[user] = dataS.User[user]
            dataS.user_valid[user] = []
            dataS.user_test[user] = []
        else:
            dataS.user_train[user] = data.User[user][:-2]
            dataS.user_valid[user] = []
            dataS.user_valid[user].append(data.User[user][-2])
            dataS.user_test[user] = []
            dataS.user_test[user].append(data.User[user][-1])

In [107]:
data_partition_with_time(dataS)

In [32]:
# model variables
num_epochs = 5
batch_size = 128
RANDOM_SEED = 100  # Set None for non-deterministic result

lr = 0.001             # learning rate
maxlen = 50            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 100     # number of units in the attention calculation
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = 20000     # number of negative examples per positive example


In [33]:
# sample negative examples
sampler = WarpSampler(dataS.user_train, dataS.usernum, dataS.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)

In [34]:
model = SASREC(item_num=dataS.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [100, 100],
               l2_reg=l2_emb,
               num_neg_test=num_neg_test)

In [36]:
with Timer() as train_time:
    t_test = model.train(dataS, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=6)


epoch: 5, test (NDCG@10: 0.8581872261916548, HR@10: 0.8692728036990333)


In [37]:
import random
from tqdm import tqdm

def get_predictions(data):
    """
        Модифицированный метод evaluation класса SASREC,
        главный результат - получаем предсказания для всех пользователей из датасета
    """
    usernum = data.usernum # max № of user
    itemnum = data.itemnum # max № of item
    train = data.user_train
    valid = data.user_valid
    test = data.user_test

    pred_dict = {}
    all_inputs = {}
    
    # насэмплим рандомных 10000 пользователей (или меньше, если их разнообразие небольшое:))
    if usernum > 10000:
        users = random.sample(range(1, usernum + 1), 10000)
    else:
        users = range(1, usernum + 1)
    
    # для каждого пользователя делаем оценку
    for u in tqdm(users, ncols=70, leave=False, unit="b"):

        if len(train[u]) < 1 or len(test[u]) < 1: # если для текущего пользователя нет ничего в train или test => continue
            continue
        # для input_seq
        seq = np.zeros([model.seq_max_len], dtype=np.int32)
        idx = model.seq_max_len - 1
        seq[idx] = valid[u][0]
        idx -= 1
        for i in reversed(train[u]): 
            seq[idx] = i
            idx -= 1
            if idx == -1: # если нет в train и valid => break
                break
        # для candidate
        rated = set(train[u]) # то, что оценил пользователь, из train'a
        rated.add(0)
        item_idx = [test[u][0]] # первым в последовательность помещаем тык из теста
        for _ in range(model.num_neg_test): # размер последовательностей получается фиксированный, задаётся в параметрах модели при инициализации
                                            # https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/sasrec_amazon.ipynb
            t = np.random.randint(1, itemnum + 1)
            while t in rated:
                t = np.random.randint(1, itemnum + 1) # генерим рандомно (?)
            item_idx.append(t)

        inputs = {}
        inputs["user"] = np.expand_dims(np.array([u]), axis=-1) # просто номер пользователя
        inputs["input_seq"] = np.array([seq]) # входная последовательность что тыкнул пользователь - ИЗ TRAIN И VALID!
        inputs["candidate"] = np.array([item_idx]) # объявки, для которых будем вычислять логиты
        all_inputs[inputs["user"][0][0]] = inputs # словарик для всех inputs

        # добавляем каждого пользователя в словарик предиктов
        pred_dict[inputs["user"][0][0]] = model.predict(inputs)

    return pred_dict, all_inputs

In [38]:
pred_dict, inputs = get_predictions(dataS)

In [41]:
# нужно связать значения логитов и кандидатов, выбрать самые вероятные значения и проверить, были ли они в тесте
# сразу расчёт precision@k
k = 5
fin_prec = 0
for key in pred_dict.keys():
  tmp = np.stack((pred_dict[key][0], inputs[key]['candidate'][0]), axis=-1)
  tmp = tmp[tmp[:, 0].argsort()] # сортировка по вероятностям (логитам)
  topk = tmp[-k:, 1].astype(int)
  # проверим, есть ли этот топk в тесте
  tmp_prec = 0
  for val in topk:
    if val in dataS.user_test[key]:
      tmp_prec += 1/k
  fin_prec += tmp_prec

fin_prec /= len(pred_dict.keys())
print("ОТВЕТ", fin_prec)

ОТВЕТ 0.19084692267230316


# OLD SAS REC

In [23]:
# create .txt file for input to model
my_data[['offer_id_enc',	'uid_enc']].to_csv('out.txt', sep="\t", header=False, index=False)

KeyError: ignored